In [4]:
import numpy as np
import math
from IPython.core.debugger import set_trace

lowerBound, upperBound = -5, 10
x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
N = len(x)

def f(x):
    #Sphere
    #Sum = 0
    #for i in range(0,N):
    #    Sum = Sum + (x[i])**2

    #Rosenbrock
    Sum = 0
    for i in range(0, N-1):
        Sum = Sum + 100 * ((x[i])**2 - x[i+1])**2 + (x[i] - 1)**2
        
    #Ackley
    #Sum1, Sum2 = 0, 0
    #for i in range(0,N):
    #    Sum1 = Sum1 + (x[i])**2
    #    Sum2 = Sum2 + math.cos(2*math.pi*x[i])
    #Sum= -20.0*math.exp(-0.2*math.sqrt((1/N)*(Sum1)))-math.exp((1/N) * Sum2) + math.e + 20
    return Sum

l = [lowerBound for i in range(10)]
u = [upperBound for i in range(10)]

In [5]:
def _e(index,size):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr

In [6]:
def Delta(x):
    Sum = 0
    #no general, only box constraints
    for i in range(0,N):
        Sum = Sum + max(x[i]-u[i],0) + max(l[i]-x[i], 0)
    return Sum

In [7]:
def better(x,y):
    Dx = Delta(x)
    Dy = Delta(y)
    if(Dx < Dy):
        return True
    elif(Dx == Dy and f(x) < f(y)):
        return True
    else:
        return False

In [8]:
def findExtremes(s):
    Ds = np.empty([N+1])
    fs = np.empty([N+1])
    
    for i in range(0, N+1):
        Ds[i] = Delta(s[i])
        fs[i] = f(s[i])
    
    arg_b = np.argmin(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_b] and i != arg_b:
            if(fs[i] < fs[arg_b]):
                arg_b = i;
                
    arg_w = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_w] and i != arg_w:
            if(fs[i] > fs[arg_w]):
                arg_w = i;
    
    #mask the worst to find the second worst
    Ds[arg_w] = 0
    
    arg_sw = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_sw] and i != arg_w:
            if(fs[i] > fs[arg_sw]):
                arg_sw = i;
    
    return arg_b,arg_w,arg_sw

In [9]:
def centroid(s,argWorst):
    return np.mean(np.delete(s, argWorst), axis = 0)

In [10]:
def SimplexLocalSearch(x,Epsilon,Lambda,M,Alpha=1,Gamma=2,Beta=-0.5):
    N = len(x)
    s = np.empty([N+1,N])
    s[0] = x
    for j in range(1,N+1):
        s[j] = x + Lambda*(u[j-1]-l[j-1])*_e(j-1,N);
    k = 0
    
    
    argBest, argWorst, argSecondWorst = findExtremes(s)
    sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
    #set_trace()
    while abs(f(sBest) - f(sWorst)) > Epsilon and k < M:
        #set_trace()
        #if(k%1000 == 0):print(f(sBest),'\n')
        r = (1+Alpha)*centroid(s,argWorst) - Alpha*sWorst
        if(better(sBest,r) and better(r,sSecondWorst)):
            s[argWorst] = r
        elif(better(r,sBest)):
            e = (1+Gamma)*centroid(s,argWorst)-Gamma*sWorst
            if(better(e,r)):
                s[argWorst] = e
            else:
                s[argWorst] = r
        else:
            c = (1+Beta)*centroid(s,argWorst)-Beta*sWorst
            if(better(c,sWorst)):
                s[argWorst] = c
            else:
                for j in range(0,N+1):
                    s[j] = (s[j] + sBest)/2

        #set_trace()
        argBest, argWorst, argSecondWorst = findExtremes(s)
        sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
        k = k + 1
    #print(f(sBest))
    return sBest, k, s, argBest, argWorst, argSecondWorst

In [21]:
def DirectionalEscape(Epsilon,EpsilonApostrophe,Lambda,M, Gamma = 2,Beta=-0.5):
    k = 0
    it = 0
    x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
    xWorse = x
    ovo_je_test = True
    while(k < M):
        #set_trace()
        x, _k, s, argBest, argWorst, argSecondWorst = SimplexLocalSearch(x,Epsilon,Lambda,M-k)
        if k == 0 and ovo_je_test:
            ovo_je_test = False
            xBest = x
        if( (k + _k)//1000 > it):
            it = (k + _k)//1000
            print("iteration:", it * 1000, "| best found value: ", f(xBest), "but x is here: ", f(x))
        k = k + _k
        if f(x) < f(xBest):
            xBest = x
            print("***NEW BEST VALUE*** iteration: ", k, "| best found value: ", f(xBest))
        first = True
        
        while(first or better(x, xWorse) ):
            first = False
            xWorse = x
            c = (1+Beta)*centroid(s,argWorst)-Beta*s[argWorst]
            x = Gamma*s[argSecondWorst] + (1-Gamma) * c
            s[argBest] = x
        
    return xBest

In [22]:
Lambda = 1
M = 100000
Epsilon = 10e-1
#set_trace()
EpsilonApostrophe = 0
print(DirectionalEscape(Epsilon,EpsilonApostrophe,Lambda,M))

***NEW BEST VALUE*** iteration:  297 | best found value:  302161.6947185616
***NEW BEST VALUE*** iteration:  391 | best found value:  291788.72670926625
***NEW BEST VALUE*** iteration:  486 | best found value:  283678.259146758
***NEW BEST VALUE*** iteration:  578 | best found value:  273760.5610910119
***NEW BEST VALUE*** iteration:  682 | best found value:  208997.86408366522
iteration: 1000 | best found value:  161370.05720326555 but x is here:  385343.5278361451
iteration: 2000 | best found value:  161370.05720326555 but x is here:  1706432.5363449778
***NEW BEST VALUE*** iteration:  2501 | best found value:  870.705918659956
***NEW BEST VALUE*** iteration:  2615 | best found value:  549.5889295409764
***NEW BEST VALUE*** iteration:  2735 | best found value:  341.0595161132424
***NEW BEST VALUE*** iteration:  2862 | best found value:  118.35145248559566
iteration: 3000 | best found value:  85.20628408590315 but x is here:  56.20917811653435
***NEW BEST VALUE*** iteration:  3053 | b

iteration: 14000 | best found value:  3.9339119764578254 but x is here:  1.269100158837292
***NEW BEST VALUE*** iteration:  14104 | best found value:  1.269100158837292
***NEW BEST VALUE*** iteration:  14241 | best found value:  1.2607858916815562
***NEW BEST VALUE*** iteration:  14374 | best found value:  1.4932085587780637
***NEW BEST VALUE*** iteration:  14505 | best found value:  1.3645287383388518
***NEW BEST VALUE*** iteration:  14639 | best found value:  1.4881574934855335
***NEW BEST VALUE*** iteration:  14773 | best found value:  1.267352038748354
***NEW BEST VALUE*** iteration:  14911 | best found value:  1.5003208712824676
iteration: 15000 | best found value:  7.304013618404918 but x is here:  1.2146211120524035
***NEW BEST VALUE*** iteration:  15043 | best found value:  1.2146211120524035
***NEW BEST VALUE*** iteration:  15181 | best found value:  0.42461059246872607
***NEW BEST VALUE*** iteration:  15306 | best found value:  1.2226430465965907
***NEW BEST VALUE*** iteratio

***NEW BEST VALUE*** iteration:  26330 | best found value:  0.4104269956783569
***NEW BEST VALUE*** iteration:  26455 | best found value:  1.1900998490242745
***NEW BEST VALUE*** iteration:  26608 | best found value:  1.5997493186805622
***NEW BEST VALUE*** iteration:  26764 | best found value:  1.1877459174584017
***NEW BEST VALUE*** iteration:  26905 | best found value:  1.228480688204324
iteration: 27000 | best found value:  4.617682961621736 but x is here:  1.2262551611762114
***NEW BEST VALUE*** iteration:  27040 | best found value:  1.2262551611762114
***NEW BEST VALUE*** iteration:  27188 | best found value:  0.4187171978785449
***NEW BEST VALUE*** iteration:  27328 | best found value:  1.2365551076427066
***NEW BEST VALUE*** iteration:  27471 | best found value:  1.2880224943915937
***NEW BEST VALUE*** iteration:  27601 | best found value:  1.2990980117322557
***NEW BEST VALUE*** iteration:  27737 | best found value:  1.0514197350921681
***NEW BEST VALUE*** iteration:  27862 | 

***NEW BEST VALUE*** iteration:  39153 | best found value:  1.084458159662357
***NEW BEST VALUE*** iteration:  39278 | best found value:  1.2307403801359604
***NEW BEST VALUE*** iteration:  39405 | best found value:  1.2190361111921202
***NEW BEST VALUE*** iteration:  39539 | best found value:  1.5749951974646137
***NEW BEST VALUE*** iteration:  39666 | best found value:  1.2610984853823202
***NEW BEST VALUE*** iteration:  39797 | best found value:  1.566233573299512
***NEW BEST VALUE*** iteration:  39943 | best found value:  1.2238916214423512
iteration: 40000 | best found value:  6.748076613974463 but x is here:  1.2268369062650732
***NEW BEST VALUE*** iteration:  40086 | best found value:  1.2268369062650732
***NEW BEST VALUE*** iteration:  40232 | best found value:  0.42563863284446707
***NEW BEST VALUE*** iteration:  40372 | best found value:  0.41512937267744093
***NEW BEST VALUE*** iteration:  40512 | best found value:  0.4097023639324341
***NEW BEST VALUE*** iteration:  40669 |

***NEW BEST VALUE*** iteration:  51527 | best found value:  1.2072028736280989
***NEW BEST VALUE*** iteration:  51669 | best found value:  1.2730343409879958
***NEW BEST VALUE*** iteration:  51809 | best found value:  1.5290166733333361
***NEW BEST VALUE*** iteration:  51944 | best found value:  1.6098487308407505
iteration: 52000 | best found value:  7.180793220683114 but x is here:  1.3018958451879548
***NEW BEST VALUE*** iteration:  52074 | best found value:  1.3018958451879548
***NEW BEST VALUE*** iteration:  52204 | best found value:  1.1484434979150384
***NEW BEST VALUE*** iteration:  52351 | best found value:  1.9036857134065506
***NEW BEST VALUE*** iteration:  52522 | best found value:  1.3602301808175248
***NEW BEST VALUE*** iteration:  52664 | best found value:  1.2760036605887655
***NEW BEST VALUE*** iteration:  52797 | best found value:  1.70191464977479
***NEW BEST VALUE*** iteration:  52933 | best found value:  1.529070833882578
iteration: 53000 | best found value:  7.504

***NEW BEST VALUE*** iteration:  64262 | best found value:  0.42960009479091443
***NEW BEST VALUE*** iteration:  64395 | best found value:  1.2793510673579587
***NEW BEST VALUE*** iteration:  64526 | best found value:  1.2890491441045993
***NEW BEST VALUE*** iteration:  64658 | best found value:  1.183248665869651
***NEW BEST VALUE*** iteration:  64788 | best found value:  1.207688023393493
***NEW BEST VALUE*** iteration:  64918 | best found value:  1.0835896590261935
iteration: 65000 | best found value:  6.879785441761677 but x is here:  1.4414397467493778
***NEW BEST VALUE*** iteration:  65057 | best found value:  1.4414397467493778
***NEW BEST VALUE*** iteration:  65191 | best found value:  1.2915284345159745
***NEW BEST VALUE*** iteration:  65321 | best found value:  1.2213423070694869
***NEW BEST VALUE*** iteration:  65454 | best found value:  1.1874819944776345
***NEW BEST VALUE*** iteration:  65594 | best found value:  1.2610257924754156
***NEW BEST VALUE*** iteration:  65730 | 

iteration: 77000 | best found value:  8.292740277747907 but x is here:  0.4173237667749495
***NEW BEST VALUE*** iteration:  77070 | best found value:  0.4173237667749495
***NEW BEST VALUE*** iteration:  77199 | best found value:  1.2298863377540805
***NEW BEST VALUE*** iteration:  77331 | best found value:  1.2013908409699572
***NEW BEST VALUE*** iteration:  77459 | best found value:  1.4936841508659022
***NEW BEST VALUE*** iteration:  77587 | best found value:  1.2954250338344826
***NEW BEST VALUE*** iteration:  77723 | best found value:  1.291554296165386
***NEW BEST VALUE*** iteration:  77870 | best found value:  1.1772502116369752
iteration: 78000 | best found value:  8.571732214332261 but x is here:  1.2648034819645868
***NEW BEST VALUE*** iteration:  78031 | best found value:  1.2648034819645868
***NEW BEST VALUE*** iteration:  78156 | best found value:  1.0208984732747288
***NEW BEST VALUE*** iteration:  78286 | best found value:  1.2655279283250347
***NEW BEST VALUE*** iteratio

***NEW BEST VALUE*** iteration:  89950 | best found value:  1.392431073598777
iteration: 90000 | best found value:  7.174304781905857 but x is here:  1.2373912027581602
***NEW BEST VALUE*** iteration:  90093 | best found value:  1.2373912027581602
***NEW BEST VALUE*** iteration:  90231 | best found value:  1.2245260997148129
***NEW BEST VALUE*** iteration:  90361 | best found value:  1.2324093029768617
***NEW BEST VALUE*** iteration:  90497 | best found value:  1.2333557747581159
***NEW BEST VALUE*** iteration:  90629 | best found value:  1.2085261593842471
***NEW BEST VALUE*** iteration:  90764 | best found value:  1.089627250710411
***NEW BEST VALUE*** iteration:  90907 | best found value:  1.3292060634747742
iteration: 91000 | best found value:  8.690601263027297 but x is here:  1.3511563880463784
***NEW BEST VALUE*** iteration:  91044 | best found value:  1.3511563880463784
***NEW BEST VALUE*** iteration:  91176 | best found value:  1.2248754743440073
***NEW BEST VALUE*** iteration